In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../')

# Exploratory Data Analysis of Block Data

Loading the new classes

In [2]:
from src.d01_data.student_data_api import StudentDataApi
from src.d01_data.block_data_api import BlockDataApi
from src.d01_data.abstract_data_api import AbstractDataApi

block_data_api = BlockDataApi()
student_data_api = StudentDataApi()
etl = AbstractDataApi()

Getting the data we will need

In [3]:
#SFHA_block_df = block_data_api.get_data(sfha=True)
block_df = block_data_api.get_data(sfha=False)
#student_df = student_data_api.get_data()
field_descriptions_df = block_data_api._cache_demographic['fields']

## SFHA Block Data

We want to know whether the SFHA dataset is redundant or it provides relevant information. We can see that the information it contains is entirely contained in the main Block dataset, so we can ignore it:

In [18]:
SFHA_fields = list(SFHA_block_df.columns)

In [19]:
SFHA_fields

['ID',
 'AREA',
 'DATA',
 'Block',
 'Block Type',
 'BlockGroup',
 'Tract',
 'County',
 'Place',
 'UnifSchool',
 'SF Analysis Neighborhood',
 'Current ESAA',
 '2010 total population count',
 'ACS 2013-17 est median HH income',
 'ACS 2013-17 est% HH below poverty lvl',
 'SFHA_Hsng',
 'SA_Hsng',
 'SFHA_ex_Sr']

In [20]:
block_df_new = block_df.filter(['ID','Block','SFHA_Hsng', "SA_Hsng"], axis=1)
SFHA_block_df_new = SFHA_block_df.filter(['ID','Block','SFHA_Hsng', "SA_Hsng"], axis=1)

merge_df = pd.merge(SFHA_block_df_new, block_df_new, on=['ID'], how='inner')
merge_df.head()

,ID,Block_x,SFHA_Hsng_x,SA_Hsng_x,Block_y,SFHA_Hsng_y,SA_Hsng_y
0,429415,60750105001001,no,no,60750105001001,no,no
1,427881,60750105001023,no,no,60750105001023,no,no
2,427720,60750106002000,no,no,60750106002000,no,no
3,423720,60750615002014,no,yes,60750615002014,no,yes
4,420222,60750611001022,no,no,60750611001022,no,no


Any difference would appear here (can be tried with any other pair of columns):

In [21]:
count = 0
L = list(merge_df["SA_Hsng_x"] == merge_df["SA_Hsng_y"])
for i in range(len(L)):
    if L[i] == False:
        print(i)
        count += 1

We are good! We do not need to use the SFHA dataset!

## Block Data

In [9]:
pd.set_option("max_rows", None)
field_descriptions_df[["Field Name", "Field Description", "Data Source"]]

,Field Name,Field Description,Data Source
0,ID,ID,assigned by GIS software
1,Area,AREA,area in square miles
2,Block,Block,2010 Census block ID
3,Block Type,Block Type,land block: zero pop or populated
4,BlockGroup,BlockGroup,2010 Census block group ID
5,Tract,Tract,2010 Census Tract ID
6,County,County,2010 Census County ID
7,Place,Place,2010 Census place (city) ID
8,UnifSchool,UnifSchool,2010 Census Unified School District ID
9,ZIP5,ZIP5,5-digit ZIP Code


Grouping the columns in useful "thematic" groups:

1. IDENTIFICATION: Reflects geographic characteristics and id numbers (Census Tracts, area, FIPS, block type)
2. CURRENT: Columns referring to current CTIP assignment and simulations of variations in the exisiting model by the district
3. POPULATION: Demographic information such as population by age and enrollment in schools, also parents educational level and language status
4. ETHNICITY: Information on ethnicity of residents and students
5. ETHNICITY_DETAILED: Breakdown of ethnicity by grade, detailed ethnic group, and year (district)
6. ETHNICITY_DETAILED_GROUP: Above data but grouped for subsequent grades
6. INCOME: Data referrent to income and wealth of block and families
7. TEST SCORES: Academic data on CST and SBAC
8. HOUSING: San Francisco and Federal Hosuing Authority information

In [10]:
#write list of the most important ones and share with district
from collections import defaultdict

field_list = list(block_df.columns)

race_words = ["Black", "African", "Filipino", "Asian", "Hisp", "White", "Samoa", "Mixed", "Pacific",
              "Decl", "Other", "DS",
              "Wht", "AALPI", "Asn", "AfAm"]

income_words = ["income", "Lunch", "poverty"]

housing_words = ["Hsng", "SFHA", "Hsg"]

testing_words = ["SBAC", "CST", "test"]


group_dict1 = defaultdict(list)

for col in field_list:
    
    idx = field_list.index(col)
    
    #The first columns are identification columns:
    if idx <= field_list.index("SF Analysis Neighborhood"):
        group_dict1["ID"].append(col)
    
    #Then there are CTIP-related columns:
    elif idx <= field_list.index("Possible change in CTIP1 status Model 2"):
        group_dict1["CURRENT"].append(col)
    
    #Now let's take all the ethnicity-related columns, starting with the most detailed breakdown:
    elif "F201" in col:
        group_dict1["ETHNICITY_DETAILED"].append(col)
        
    #The second level of detailed ethnicity (grouped for a few grades):
    elif col[6] == "-" and "stu" not in col:
        group_dict1["ETHNICITY_DETAILED_GROUP"].append(col)
        
    #Non-detailed ethnicity:
    elif any(word in col for word in race_words):
        group_dict1["ETHNICITY"].append(col)
    
    #Income-related columns:
    elif any(word in col for word in income_words):
        group_dict1["INCOME"].append(col)

    #Housing-related columns:
    elif any(word in col for word in housing_words):
        group_dict1["HOUSING"].append(col)

    #Testing-related columns:
    elif any(word in col for word in testing_words):
        group_dict1["TEST"].append(col)
        
    #All others are general demographic info:
    else:
        group_dict1["DEMOGRAPHIC"].append(col)

In [11]:
from itertools import chain

used = list(chain.from_iterable(list(group_dict1.values())))
for col in field_list:
    if col not in used:
        print(col)

In [17]:
group_dict1["TEST"]

['num of SBAC L1 scores 4-9 2015-18',
 'num of SBAC L2 scores 4-9 2015-18',
 'num of SBAC L3 scores 4-9 2015-18',
 'num of SBAC L4 scores 4-9 2015-18',
 'ttl num 4-9 test takers 2015-18',
 'CST ELA test takers gr4-9 2011-2014',
 'average CST ELA score 2011-14',
 'CST ELA test takers gr4-9 2006-10',
 'avg CST ELA score 2006-10',
 'CSTELA test takers 2006-10',
 'avg CSTELA score 2006-10']

UPDATE: Testing to get the dictionary with the method in the block data api class:

In [4]:
group_dict_2 = block_data_api.get_classification()

In [8]:
group_dict_2

defaultdict(list,
            {'ID': ['ID',
              'Area',
              'DATA',
              'Block',
              'Block Type',
              'BlockGroup',
              'Tract',
              'County',
              'Place',
              'UnifSchool',
              'ZIP5',
              'SF Analysis Neighborhood'],
             'CURRENT': ['Current ESAA',
              'Scenario 1',
              'Scenario 2',
              'CTIP_2013 assignment',
              'Possible change in CTIP1 status Model 2'],
             'DEMOGRAPHIC': ['2010 total population count',
              '2010 total pop aged 18+',
              '2010 pop less than 18 years old',
              'ACS 2013-17 % aged 25+ <HS diploma',
              'ACS 2013-17 % aged 25+ HS diploma',
              'ACS 2013-17 % aged 25+ some coll no BA',
              'ACS 2013-17 % aged 25+ with Bachelors',
              'ACS 2013-17 % aged 25+ grad or prof degr',
              'ACS 2013-17 est% HH with children <17',


Search for a specific description:

In [44]:
name = "SFHA_ex_Sr"
field_descriptions_df.loc[field_descriptions_df["Field Name"] == name, "Field Description"].iloc[0]

'Does block contain non-senior SFHA units? (yes/no)'

Verifying if the field names and block data all match:

In [7]:
block_fields = list(block_df.columns)

In [8]:
field_fields = list(field_descriptions_df["Field Name"].values)

In [9]:
def Diff(li1, li2):
    return list(set(li1) - set(li2))

Diff(block_fields, field_fields)

['DATA']

In [10]:
Diff(field_fields, block_fields)

['Free Reduced Lunch']

The FRL column is a mystery, and so is the DATA in the block dataset. For the other two columns columns we notice there is a year discrepancy. Should check with Henry.

UPDATE: Included that pre-processing in the block data API for the area formatting. AREA doesn't appear in the above lsit anymore.

In [12]:
name = "CSTELA test takers 2006-10"
field_descriptions_df.loc[field_descriptions_df["Field Name"] == name, "Field Description"].iloc[0]

'Number of CST ELA test takers K-8 2006-2010'

We notice that the FIELDS dataframe has the wrong name for the CST ELA (the above result shows that 2006-2010 was the timespan, but the previous column title was 2006-2014). I now fixed it upon initialization in the api.

## Students vs. Residents

In [42]:
group_dict1["ETHNICITY"]

['2010 Hispanic',
 '2010 non-Hisp African American',
 '2010 non-Hisp Amer Indian Alaska Native',
 '2010 non-Hisp Asian',
 '2010 non-Hisp Pacific Islander',
 '2010 non-Hisp White',
 '2010 non-Hisp Other',
 '2010 non-Hispanic Mixed Race',
 '2010 pop 18+ Hispanic',
 '2010 pop 18+ non-Hisp African American',
 '2010 pop 18+ non-Hisp Amer Ind AK native',
 '2010 pop 18+ non-Hisp Asian',
 '2010 pop 18+ non-Hisp Pacific Islander',
 '2010 pop 18+ non-Hisp White',
 '2010 pop 18+ non-Hisp Other',
 '2010 pop 18+ non-Hisp Mixed Race',
 '2010 pop <18 Hispanic',
 '2010 pop <18 non-Hisp African American',
 '2010 pop <18 non-Hisp Amer Ind AK native',
 '2010 pop <18 non-Hisp Asian',
 '2010 pop <18 non-Hisp Pacific Islander',
 '2010 pop <18 non-Hisp White',
 '2010 pop <18 non-Hisp Other',
 '2010 pop <18 non-Hisp Mixed Race',
 'AALPI all TK5 stu 2017',
 'non-AALPI all TK5 stu 2017',
 'DS or ML all TK5 stu 2017',
 'All Others all TK5 stu 2017',
 'AALPI reg TK5 stu 2017',
 'non-AALPI reg TK5 stu 2017',
 'DS 

In [35]:
name = "all TK-5 2017 stu"
field_descriptions_df.loc[field_descriptions_df["Field Name"] == name, "Field Description"].iloc[0]

'Fall 2017 TOTAL TK-5 student residents'

In [22]:
name = 'ACS 2013-17 est% aged 5-14 public sch'
field_descriptions_df.loc[field_descriptions_df["Field Name"] == name, "Field Description"].iloc[0]

'ACS 2013-17 est % of children aged 5-14 enrolled in public school'

Measuring the correlation between enrollment in public schools as measured by ACS vs as measured by the district:

In [37]:
M = block_df[[ "ACS 2013-17 est% aged 5-14 public sch", "K8stu1517", "number of K8 residents fall 2017"]].to_numpy()

print(np.corrcoef(np.transpose(M)))

[[1.         0.19633266 0.19531274]
 [0.19633266 1.         0.99049856]
 [0.19531274 0.99049856 1.        ]]


Last two columns, both provided by the district, correlate which is a good sanity check (only difference is time scale). But ACS estimate seems not to be correlating with the other two

In [38]:
block_df[["ACS 2013-17 est% aged 5-14 public sch", "K8stu1517"]]

,ACS 2013-17 est% aged 5-14 public sch,K8stu1517
0,0.549020,0
1,0.549020,130
2,0.745562,0
3,0.745562,0
4,0.745562,0
...,...,...
7318,0.549020,0
7319,0.549020,0
7320,0.549020,0
7321,0.549020,0


Seems like many blocks have a zero value for the district data! We can see if the correlation gets better when the value is not zero:

In [54]:
temp_df = block_df[["ACS 2013-17 est% aged 5-14 public sch", "K8stu1517", "number of K8 residents fall 2017"]]

zero_cond = (temp_df != 0).all(axis=1)
new_df = temp_df.loc[zero_cond]

In [55]:
print(np.corrcoef(np.transpose(new_df.to_numpy())))

[[1.         0.28607965 0.28486351]
 [0.28607965 1.         0.98795362]
 [0.28486351 0.98795362 1.        ]]


Still not the best. Maybe we need to understand better how this data is computed.